# Uniform

In [1]:
!pip install DeepCache

In [1]:
prompt='A man is climbing a mountain'

In [2]:
import torch

# Loading the original pipeline
from diffusers import StableDiffusionXLPipeline
pipe = StableDiffusionXLPipeline.from_pretrained('g:code/model/stable-diffusion-xl-base-1.0', torch_dtype=torch.float16,variant='fp16').to("cuda:0")


A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "e:\conda_data\envs\heavy_daily\lib\site-packages\xformers\__init__.py", line 57, in _is_triton_available
    import triton  # noqa
ModuleNotFoundError: No module named 'triton'


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
# 使用xformers进行内存优化
pipe.enable_xformers_memory_efficient_attention()
# 使用sliced attention减少内存占用
pipe.enable_attention_slicing(slice_size=1)
# 启用模型CPU卸载以节省GPU内存
pipe.enable_model_cpu_offload()
# 使用flash attention加速计算
pipe.enable_vae_slicing()  # 启用VAE切片以减少内存使用


In [5]:
pipe.safety_checker = None  # 禁用安全检查器以节省内存

In [6]:
# Import the DeepCacheSDHelper
from DeepCache import DeepCacheSDHelper
helper = DeepCacheSDHelper(pipe=pipe)
helper.set_params(
    cache_interval=3,
    cache_branch_id=0,
)
helper.enable()

# Generate Image
deepcache_image = pipe(
        prompt,
        output_type='pt'
).images[0]
helper.disable()

  0%|          | 0/50 [00:00<?, ?it/s]

In [13]:
torch.cuda.empty_cache()

In [15]:
# 对比生成15张图片使用deepcache和使用原始pipeline的生成时间
import time
from tqdm import tqdm
# 定义测试次数
num_images = 5
'''# 测试原始pipeline的生成时间
original_times = []
print("测试原始pipeline生成时间...")
for _ in tqdm(range(num_images)):
    start_time = time.time()
    _ = pipe(prompt, output_type='pt').images[0]
    end_time = time.time()
    torch.cuda.empty_cache()
    original_times.append(end_time - start_time)'''
# 测试DeepCache的生成时间
helper.enable()
deepcache_times = []
print("测试DeepCache生成时间...")
for _ in tqdm(range(num_images)):
    start_time = time.time()
    _ = pipe(prompt, output_type='pt').images[0]
    end_time = time.time()
    torch.cuda.empty_cache()
    deepcache_times.append(end_time - start_time)
helper.disable()
# 计算平均时间
#avg_original = sum(original_times) / len(original_times)
avg_deepcache = sum(deepcache_times) / len(deepcache_times)
# 输出结果
#print(f"原始pipeline平均生成时间: {avg_original:.4f}秒")
print(f"DeepCache平均生成时间: {avg_deepcache:.4f}秒")
#print(f"加速比: {avg_original/avg_deepcache:.2f}x")

测试DeepCache生成时间...


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

 20%|██        | 1/5 [00:22<01:29, 22.27s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

 80%|████████  | 4/5 [00:45<00:09,  9.71s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:53<00:00, 10.62s/it]

DeepCache平均生成时间: 10.6160秒


# Non-Uniform
代码只对视频生成做了Non-uniform 实现